# Pràctica 1 de Processat del Llenguatge Natural:
# Ús d'expressions regulars en Python
En aquesta pràctica utilitzarem el mòdul `re` de Python i la llibreria Pandas per a tractar amb expressions regulars.

### Noms:
*David Sanchis Morales* \
*Raul Gómez Beteta*

## Cerca de patrons
En el mòdul `re` hi ha 4 mètodes per a buscar un patró de text: `re.match`, `re.search`, `re.findall` i `re.finditer`


In [85]:
import re

texto = '"Ethics are built right into the ideals and objectives of the United Nations" \
#UNSG @NY @UN_Women Society for Ethical Culture bit.ly/2guVelr'

print(re.match(r'@[A-Z]+', texto))

None


`re.match` només cerca al inici de la cadena, `re.search` cerca la primera aparició del patró en el text.

In [86]:
print(re.match(r'#[A-Z]+', texto))

None


In [87]:
print(re.search(r'@[\w]+', texto))

<re.Match object; span=(84, 87), match='@NY'>


In [88]:
print(re.search(r'@[A-Z]+', texto))

<re.Match object; span=(84, 87), match='@NY'>


`re.match` i `re.search` tornen un objecte `Match` que té diversos mètodes. El mètode `group()` conté el patró trobat.

In [47]:
match = re.search(r'#[A-Z]+', texto)
print('span:',match.span())
print('start:',match.start())
print('end:',match.end())
print('group:',match.group())

span: (78, 83)
start: 78
end: 83
group: #UNSG


Per a buscar totes les aparicions d'un patró en el text usem la funció `re.findall`

In [48]:
print(re.findall(r'@[\w_]+', texto))

['@NY', '@UN_Women']


## Patrons complexos
Exemple: volem detectar qualsevol data en un text, però la data pot seguir diferents patrons. Cal definir un patró amb totes les possibilitats...

In [49]:
fechas = '23/9/2010, 23/09/2010, 23/9-10, 23-9-2010, 2/9/2010'

In [50]:
re.findall(r'\d{2}/\d{1,2}/\d{4}', fechas) #captura sólo las fechas con formado dd/(m)m/aaaa

['23/9/2010', '23/09/2010']

In [51]:
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', fechas) #captura todos los formatos de fecha posibles

['23/9/2010', '23/09/2010', '23/9-10', '23-9-2010', '2/9/2010']

In [52]:
texto = 'El avión salió el 5/10/12 de Caracas'
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', texto)

['5/10/12']

### Exercici 1
Com podem detectar *totes* les dates d'aquest text?  
*Ajuda: Utilitza l'operador* OR (`|`) *per a especificar dos patrons alternatius*

In [53]:
texto = '''Francisco nació el 28/3/78, se casó el 20 de mayo del 98 y tuvo 2 hijos,
el primero nació el 3-10-2001 y el segundo el 2 de junio de 2004'''

In [54]:
re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2} de \w+ del? \d{2,4}', texto)


['28/3/78', '20 de mayo del 98', '3-10-2001', '2 de junio de 2004']

## Captura de grups
Podem buscar un patró compost però només recuperar una part específica d'aquest.

In [89]:
texto = '''Ingredientes de Arroz con Leche:
- 200 gramos de arroz
- 150 gramos de azúcar
- Un litro de leche entera
- Dos ramas de canela
- Piel de un limón
- Canela molida
- 50 gramos de mantequilla (Opcional)
'''

*Quants gr de sucre té la recepta?*  
Podem buscar tots els números del text:

In [56]:
re.findall(r'\d+', texto)

['200', '150', '50']

Podem buscar el patró dels grams de sucre

In [57]:
re.findall(r'\d+ gramos de azúcar', texto)

['150 gramos de azúcar']

Si d'aquest patró volem retornar només la part corresponent al número l'assignem a un grup:

In [58]:
re.findall(r'(\d+) gramos de azúcar', texto)

['150']

Podem definir diversos grups en la captura i recuperar-los com una tupla de textos.  
*P. ex. Quants grams de cada ingredient tenim?*

In [59]:
re.findall(r'(\d+) gramos de (\w+)', texto)

[('200', 'arroz'), ('150', 'azúcar'), ('50', 'mantequilla')]

Fins i tot podem assignar un nom a cada grup. L'objecte retornat crea un diccionari amb els grups trobats.

In [60]:
matches = re.search(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto)
print(matches.groups())
matches.groupdict()

('200', 'arroz')


{'gramos': '200', 'ingrediente': 'arroz'}

In [61]:
matches.group('gramos') #alternativamente matches['gramos']

'200'

In [62]:
matches.group('ingrediente') #alternativamente matches['ingrediente']

'arroz'

Per a buscar totes les repeticions d'un patró amb grups enumerats no podem usar `re.findall` sinó que hem d'usar la versió iterativa `re.finditer` que retorna un *iterable* sobre objectes de tipus `re.Match`

In [63]:
matches = list(re.finditer(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto))
matches

[<re.Match object; span=(35, 54), match='200 gramos de arroz'>,
 <re.Match object; span=(57, 77), match='150 gramos de azúcar'>,
 <re.Match object; span=(164, 188), match='50 gramos de mantequilla'>]

In [64]:
matches[0].groupdict()

{'gramos': '200', 'ingrediente': 'arroz'}

Podem crear una *list comprehension* recorrent sobre el resultat de `re.findall`:

In [65]:
[(ingredientes, gramos) for (gramos, ingredientes) in re.findall(r'(?P<gramos>\d+) gramos de (?P<ingrediente>\w+)', texto)]

[('arroz', '200'), ('azúcar', '150'), ('mantequilla', '50')]

### Exercici 2
Genera un diccionari de Python amb els grams de cada ingredient a partir de la llista `matches`. El resultat ha de ser:  
```python
{'arroz': '200', 'azúcar': '150', 'mantequilla': '50'}
```

In [66]:
diccionario = dict()

for objeto in matches:
    diccionario[objeto['ingrediente']] = objeto['gramos']
    
diccionario

{'arroz': '200', 'azúcar': '150', 'mantequilla': '50'}

### Exercici 3
Crea una llista amb tots els valors de `span()` dels patrons trobats en `matches`, guardats com tuples.

In [67]:
lista = list()

for objeto in matches:
    lista.append(objeto.span())

lista

[(35, 54), (57, 77), (164, 188)]

### Exercici 4

Donat el següent text:

In [68]:
texto = 'Some authors like Jason Foster (y.foster@abcd.com), R. Davis (rdavis22@www.uk) and Charlotte Williams (ch_williams@usa.gov) observed that...'

#### Ex 4.1
Extrau tots els noms i cognoms de persones d'aquest text com una llista de diccionaris amb les claus `nom` i `cognom`. Converteix aqueixa llista en un DataFrame de Pandas.

In [69]:
import pandas as pd
matches = list(re.finditer(r'(?P<nom>[A-Z]\w+|[A-Z]\.?) (?P<cognom>[A-Z]\w+)',texto))

lista = list()

for objeto in matches:
    lista.append(objeto.groupdict())

df = pd.DataFrame(lista)
df



,nom,cognom
0,Jason,Foster
1,R.,Davis
2,Charlotte,Williams


#### Ex. 4.2
Extrau tots els emails del text anterior com una llista.

In [70]:
re.findall(r'\w+[\.\_]?\w+@\w+\.\w+',texto)


['y.foster@abcd.com', 'rdavis22@www.uk', 'ch_williams@usa.gov']

## Ús de non-capturing groups
Quan usem un grup per a definir un patró complex però volem capturar tota l'expressió usem un *non-capturing group*:

In [71]:
texto = "entre el 15 de agosto y el 20 de septiembre, no el 15 de 2020"
re.findall(r'\d+ de (?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|\w+bre\b)', texto)

['15 de agosto', '20 de septiembre']

### Exercici 5

Crea un patró RegEx per a detectar totes les URL que pertanguen a un domini `.es` del següent text:  
Ajuda: el patró ha de buscar el text `.es` darrere d'un patró repetit `algo.` (una o més vegades) , seguit d'un patró opcional de tipus `/algo/algo_más`

In [72]:
texto = "amazon.com amazon.es google.es/shopping aliexpress.com.es www.elcorteingles.es"

In [73]:
re.findall(r'[\w+\.]+\.es(?:[/\w+]+)',texto)

['google.es/shopping']

## Substitució de patrons

La funció `re.sub()` permet substituir text capturat per una expressió regular.  

### Exercici 6

Substitueix en el següent text els imports expressats com a `$valor` a la forma `valor$`  
Nota: el símbol `$` té un significat especial en RegEx pel que cal escapar-lo com `\$`

In [74]:
texto = "El coste total fue de $320, repartidos en $225.7 en comida y $94.3 en bebida"

In [75]:
re.sub(r'\$(?P<valor>\d+\.?\d*)', r'\g<valor>$', texto)

'El coste total fue de 320$, repartidos en 225.7$ en comida y 94.3$ en bebida'

## Ús de RegEx amb objectes de Pandas
 Si tenim un objecte `Series` de Pandas podem aplicar les funcions de text o de cerca d'expressions regulars sobre el contingut de cada element (en un `DataFrame` cada columna és un objecte `Series`)

In [91]:
import pandas as pd

frases = ["Tengo cita con el doctor a las 2:45.", 
          "el martes llegaré a las 11:30.",
          "No puedo, tengo un partido a las 7:00.",
          "Nos vemos el jueves 7 a las 8:30.",
          "El tren sale a las 9:15 y llega a las 11:35."]

df = pd.DataFrame(frases, columns=['texto'])
df

,texto
0,Tengo cita con el doctor a las 2:45.
1,el martes llegaré a las 11:30.
2,"No puedo, tengo un partido a las 7:00."
3,Nos vemos el jueves 7 a las 8:30.
4,El tren sale a las 9:15 y llega a las 11:35.


### Captura de grups:

In [93]:
# busca las ocurrencias de hora:minuto en los textos
df['texto'].str.findall(r'(\d{1,2}):(\d{2})')

0              [(2, 45)]
1             [(11, 30)]
2              [(7, 00)]
3              [(8, 30)]
4    [(9, 15), (11, 35)]
Name: texto, dtype: object

### Captura de grups numerats:

In [78]:
# extrae todas las apariciones de tiempo y separa horas y minutos
df['texto'].str.extractall(r'(?P<tiempo>(?P<hora>\d?\d):(?P<minutos>\d\d))')

tiempo hora minutos
  match                    
0 0       2:45    2      45
1 0      11:30   11      30
2 0       7:00    7      00
3 0       8:30    8      30
4 0       9:15    9      15
  1      11:35   11      35

### Exercici 7
Extrau totes les dates dels següents textos, primer de manera completa i després separant dia, mes i any

In [79]:
fechas = ["Tengo cita con el doctor el 3/10", 
          "Juan nació el 28/3/78",
          "Su primer hijo nació el nació el 3-10-2001 y el segundo el 10-1-2003",
          "El 8/1/1998 se fué de viaje a Praga"]

df = pd.DataFrame(fechas, columns=['texto'])
df

,texto
0,Tengo cita con el doctor el 3/10
1,Juan nació el 28/3/78
2,Su primer hijo nació el nació el 3-10-2001 y e...
3,El 8/1/1998 se fué de viaje a Praga


In [80]:
df['texto'].str.extractall(r'(?P<fecha>(?P<dia>\d{1,2})[/-](?P<mes>\d{1,2})[/-]?(?P<año>\d{0,4}))')


fecha dia mes   año
  match                         
0 0           3/10   3  10   NaN
1 0        28/3/78  28   3    78
2 0      3-10-2001   3  10  2001
  1      10-1-2003  10   1  2003
3 0       8/1/1998   8   1  1998

### Exercici 8 
A partir de la següent llista de Tuits, genera un `DataFrame` amb les següents columnes:
 - `Tweet`: text del Tuit
 - `Menciones`: llista amb les mencions de cada Tuit
 - `Hashtag`: llista amb els hashtags de cada Tuit

In [135]:
tweets = [
    '@Yulian_Poe @guillermoterry1 Ah. mucho más por supuesto! solo que lo incluyo. Me habías entendido mal',
    'Se ha terminado #Rio2016 Lamentablemente no arriendo las ganancias al pueblo brasileño por la penuria que les espera',
    '@Yosmath @Planeta87Radio  #Incantus  Genial ya estoy conectada',
    '@seestrena #seestrenasietevidas Sería un gato tipo Garfield porqué soy un poco vago y porqué me encanta la lasaña!!',
    'Hoy toca escuchar a #MiguelRios con otros grandes artistas...  #Insurrección...'
]

In [166]:
lista = list()

for tweet in tweets:
    diccionario = dict()
    diccionario['Tweet'] = re.findall(r'.+',tweet)
    diccionario['Menciones'] = re.findall(r'(?P<Menciones>@[^ ]+)',tweet)
    diccionario['Hashtag'] = re.findall(r'(?P<Hashtag>#[^ ]+)',tweet)
    lista.append(diccionario)

df = pd.DataFrame(lista)
df


,Tweet,Menciones,Hashtag
0,[@Yulian_Poe @guillermoterry1 Ah. mucho más po...,"[@Yulian_Poe, @guillermoterry1]",[]
1,[Se ha terminado #Rio2016 Lamentablemente no a...,[],[#Rio2016]
2,[@Yosmath @Planeta87Radio #Incantus Genial y...,"[@Yosmath, @Planeta87Radio]",[#Incantus]
3,[@seestrena #seestrenasietevidas Sería un gato...,[@seestrena],[#seestrenasietevidas]
4,[Hoy toca escuchar a #MiguelRios con otros gra...,[],"[#MiguelRios, #Insurrección...]"


### Exercici 9

A partir de la següent recepta genera un dataframe amb les següents columnes:  
 - `Ingredient`: nom de l'ingredient  
 - `Quantitat`: valor numèric de la quantitat  
 - `Unitats`: unitat en la qual s'expressa la quantitat  

 *Nota*: `Quantitat` i `Unitats` són opcionals. Considera com a possibles unitats 'gr', 'grams', 'litre', 'litres','l' i 'ml'


In [167]:
texto = """
200 gr de arroz redondo.
1 litro de leche entera.
100 gr de azúcar.
2 ramas de canela.
La cáscara de un limón.
2 ml de esencia de vainilla.
50 gramos de mantequilla (Opcional).
"""

In [189]:
receta = re.findall(r'(.+)',texto)
df = pd.DataFrame(receta,columns = ['texto'])
df['texto'].str.extractall(r'(?P<Quantitat>\d+) (?P<Unitats>gr|gramos|litro|litros|l|ml|ramas) de (?P<Ingredient>\w[ \w]*)')


,,Quantitat,Unitats,Ingredient
,match,,,
0,0,200,gr,arroz redondo
1,0,1,litro,leche entera
2,0,100,gr,azúcar
3,0,2,ramas,canela
5,0,2,ml,esencia de vainilla
6,0,50,gramos,mantequilla
